# **Optimal Wifi-Hotspot Positioning Notebook**

This notebook performs analysis on architectural plans, particularly focusing on identifying zones, walls, and other features. It then applies graph theory to optimize the placement of WiFi hotspots.



Optimal WiFi-Hotspot Positioning - QUBO/Ising ---- Repo is available here : https://github.com/samgr55/OptimalWiFi-HotspotPositioning_QUBO-Ising

# **Importing Libraries**


The script starts by importing various libraries for image processing, machine learning, geometric operations, and more.                                    
Some of the notable libraries used are:
*   **cv2** (OpenCV for image processing)
*   **skimage.metrics** (for structural similarity calculation)*italicized text*
*   **matplotlib** (for visualization)
*   **shapely** (for geometric operations)
*   **dimod** (for solving optimization problems)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dimod import BinaryQuadraticModel, SimulatedAnnealingSampler, BINARY
import networkx as nx
import numpy as np
from sklearn.svm import SVC
import joblib
import numpy as np
import cv2
from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt
import os
from shapely.geometry import Polygon
from shapely.affinity import scale, translate
from shapely.geometry import LinearRing, LineString, MultiPoint, Point, MultiLineString, MultiPolygon
from sklearn.cluster import DBSCAN
from collections import defaultdict
import math
from shapely.ops import unary_union
#import dynex
import dimod

# **Architectural Plan Image Classification**

***Determine if the input image is an architectural plan or not***

### **preprocessing test image**
The function ***preprocess_test_img*** is defined to preprocess a new image. The function loads the reference architectural plan image, resizes the new image to match its dimensions, calculates the SSIM score, reshapes and scales the score using a trained scaler.

In [ ]:
def preprocess_test_img(image_path, scaler):
    reference_architectural_plan_path = "Dataset/4.png"
    # reference_architectural_plan_path = "datasets/FPOP-dataset/OWHP/4.png"
    reference_architectural_plan = cv2.imread(reference_architectural_plan_path)
    new_image = cv2.imread(image_path)
    new_image_resized = cv2.resize(new_image, (1167, 875))
    similarity_score = ssim(new_image_resized, reference_architectural_plan, channel_axis=2)
    similarity_score_reshaped = similarity_score.reshape(1, -1)
    similarity_score_scaled = scaler.transform(similarity_score_reshaped)
    return similarity_score_scaled

### **Loading the pre-trained SVM model**
 The function ***load_architectural_plan_model*** load a saved Support Vector Machine (SVM) model for architectural plan classification using preprocessed similarity scores between images. The script involves preprocessing the input image to calculate a similarity score using Structural Similarity Index (SSIM), scaling the score using a previously trained scaler, and then making predictions using the loaded SVM model.

#### NOTE: NOT used in this example

In [ ]:
def load_architectural_plan_model(new_image_path):
    # Load the saved SVM model
    loaded_model = joblib.load("svm_arch_plan_model.joblib")
    scaler = joblib.load("scaler.joblib")

    # Preprocess the new image if needed (resize, normalize, etc.)
    similarity_score_scaled =preprocess_test_img(new_image_path,scaler)
    # Predict using the loaded SVM model
    prediction = loaded_model.predict(similarity_score_scaled)

    return prediction

# **IMG2GRAPH**


***Convert the architectural plan image into a graph representation.***




### **Color Definitions and Tolerances**
Define color palettes, mapping colors to categories, and tolerances for color matching. These are used to identify different zones or areas within the architectural plan based on color information.

In [ ]:
palette_colors = {
    ( 229, 242,244): "corridor",
    (135, 216, 208): "Terrace",
    (214, 216,234 ): "kitchen",
    (171, 244, 253): "Bedroom",
    (252, 233,205 ): "Bathroom",
    (189, 222, 249): "store",
    (30, 225, 255): "entrance",
    (79, 79, 79): "exterior_wall",
    (128, 128, 128): "interior_wall"
}
tolerances = {
     ( 229, 242,244): 0.01,
     (135, 216, 208): 0.01,
     (214, 216,234 ): 0.01,
     (171, 244, 253): 0.01,
     (252, 233,205 ): 0.01,
     (189, 222, 249): 0.01,
     (30, 225, 255): 0.1,
     (79, 79, 79): 0.001,
     (128, 128, 128): 0.001
}
category_colors = {
    "corridor": (244, 242, 229),
    "Terrace": (208, 216, 135),
    "kitchen": (234, 216, 214),
    "Bedroom": (253, 244, 171),
    "Bathroom": (205, 233, 252),
    "store": (249, 222, 189),
    "entrance": (255, 225, 30),
    "exterior_wall":(79, 79, 79),
    "interior_wall": (128, 128, 128),
    "reinforcement_concrete_interior_wall": (255, 0, 0),
    "opening":(0,0,255)
}

### **Categorizing Colors**
The function **categorize_color** matches a given color to the nearest predefined color in the palette, helping to categorize different zones in the image based on their colors.

In [ ]:
def categorize_color(color):
    nearest_color = min(palette_colors, key=lambda c: np.linalg.norm(np.array(c) - np.array(color)))
    return palette_colors[nearest_color]

### **Edge Detection and Walls Baselines Extraction**
Define functions to detect edges and extract lines from the architectural plan image. These lines could represent walls or other structures. The extracted lines are grouped using techniques like DBSCAN and cleaned up to remove short or overlapping lines.

In [ ]:
def ext_line_wall(image, line_thickness=3):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret,thresh=cv2.threshold(gray,90,255,cv2.THRESH_BINARY_INV)
    # cv2.imshow('Thresh', thresh)
    edges = cv2.Canny(thresh, threshold1=30, threshold2=200)
    kernel = np.ones((line_thickness, line_thickness), np.uint8)
    dilated_edges = cv2.dilate(edges, kernel, iterations=0)
    contours, _ = cv2.findContours(dilated_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    min_contour_area = line_thickness * 10  
    filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]

    img = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2RGB)
    result = img.copy()
    return result,filtered_contours

def extract_h_lines(dilated):
    edges = cv2.Canny(dilated, threshold1=10, threshold2=255)
    kernel= np.ones((5,5), np.uint8)
    dilated_edges = cv2.dilate(edges, kernel, iterations=-3)
    # Apply Hough Line Transform to detect vertical lines
    lines = cv2.HoughLinesP(dilated_edges, 0.375, np.pi / 180, threshold=25, minLineLength=3, maxLineGap=17)
    line_endpoints = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        line_endpoints.append((x1, y1, x2, y2))
    # Apply DBSCAN to cluster similar lines
    epsilon = 20 
    min_samples = 2 
    dbscan = DBSCAN(eps=epsilon, min_samples=min_samples).fit(line_endpoints)


    clustered_lines = defaultdict(list)
    for i, label in enumerate(dbscan.labels_):
        clustered_lines[label].append(line_endpoints[i])
    for cluster_id, lines_in_cluster in clustered_lines.items():
        if cluster_id == -1:
            continue  # Skip noise points
        average_line = np.mean(lines_in_cluster, axis=0, dtype=int)
        x1, y1, x2, y2 = average_line
        # cv2.line(blank, (x1, y1), (x2, y2), (0, 0, 255), 1)
    return (clustered_lines)

def extract_v_lines(dilated):
    edges = cv2.Canny(dilated, threshold1=5, threshold2=255)
    kernel= np.ones((3,3), np.uint8)
    dilated_edges = cv2.dilate(edges, kernel, iterations=1)
    lines = cv2.HoughLinesP(dilated_edges, 0.115899, np.pi / 180, threshold=11 ,minLineLength=3, maxLineGap=1)
    line_endpoints = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        line_endpoints.append((x1, y1, x2, y2))
    # Apply DBSCAN to cluster similar lines
    epsilon = 20 
    min_samples = 2  
    dbscan = DBSCAN(eps=epsilon, min_samples=min_samples).fit(line_endpoints)
    
    clustered_lines = defaultdict(list)
    for i, label in enumerate(dbscan.labels_):
        clustered_lines[label].append(line_endpoints[i])
    for cluster_id, lines_in_cluster in clustered_lines.items():
        if cluster_id == -1:
            continue  # Skip noise points
        average_line = np.mean(lines_in_cluster, axis=0, dtype=int)
        x1, y1, x2, y2 = average_line
        # cv2.line(blank, (x1, y1), (x2, y2), (10,255, 20), 1)
    return (clustered_lines)

def remove_short_overlapping_lines(clustered_lines, min_overlap_distance=1, min_line_length=1):
    new_clustered_lines = defaultdict(list)
    for cluster_id, lines_in_cluster in clustered_lines.items():
        if cluster_id == -1:
            continue
        new_lines_in_cluster = []
        for line in lines_in_cluster:
            x1, y1, x2, y2 = line
            line_length = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
            is_short = line_length < min_line_length
            is_overlapping = any(np.abs(y1 - y2) < min_overlap_distance for _, _, y1, y2 in lines_in_cluster)
            if not is_short and not is_overlapping:
                average_line = np.mean(lines_in_cluster, axis=0, dtype=int)
                new_lines_in_cluster.append(average_line)
        if new_lines_in_cluster:
            new_clustered_lines[cluster_id] = new_lines_in_cluster
    return new_clustered_lines

def remove_close_lines(clustered_lines, max_distance=15, max_length_difference=25):
    new_clustered_lines = defaultdict(list)

    def calculate_distance(line1, line2):
        x1, y1, x2, y2 = line1
        nx1, ny1, nx2, ny2 = line2
        return np.sqrt((x1 - nx1)**2 + (y1 - ny1)**2)

    for cluster_id, lines_in_cluster in clustered_lines.items():
        if cluster_id == -1:
            continue
        merged_lines = []
        used_indices = set()
        for i, current_line in enumerate(lines_in_cluster):
            if i in used_indices:
                continue
            x1, y1, x2, y2 = current_line
            close_lines = [current_line]
            used_indices.add(i)
            for j, next_line in enumerate(lines_in_cluster[i+1:], start=i+1):
                if j in used_indices:
                    continue
                nx1, ny1, nx2, ny2 = next_line
                if calculate_distance(current_line, next_line) < max_distance:
                    if abs(y1 - ny1) < max_length_difference or abs(y2 - ny2) < max_length_difference:
                        close_lines.append(next_line)
                        used_indices.add(j)
            merged_points = np.mean(close_lines, axis=0, dtype=int)
            merged_line = (merged_points[0], merged_points[1], merged_points[2], merged_points[3])
            merged_lines.append(merged_line)
        new_clustered_lines[cluster_id] = merged_lines
    return new_clustered_lines

def int_line_walls(image, line_thickness=3):
    img = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2RGB)
    cnt_img =cv2.cvtColor(img.copy(), cv2.COLOR_BGR2RGB)
    walls=cnt_img
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret,thresh=cv2.threshold(gray,10,255,cv2.THRESH_BINARY_INV)
    # cv2.imshow('thresh',thresh)
    kernel_H = np.ones((1,5), np.uint8)
    dilated_edges_h = cv2.dilate(thresh, kernel_H, iterations=3)
    # cv2.imshow('h',dilated_edges_h)
    kernel_V = np.ones((7, 1), np.uint8)
    dilated_edges_v = cv2.dilate(thresh, kernel_V, iterations=-3)
    # cv2.imshow('v',dilated_edges_v)
    h_lines = extract_h_lines(dilated_edges_h)
    v_lines = extract_v_lines(dilated_edges_v)
    h_lines = remove_short_overlapping_lines(h_lines)
    v_lines = remove_short_overlapping_lines(v_lines)
    h_lines = remove_close_lines(h_lines)
    v_lines = remove_close_lines(v_lines)
    merged_lines = defaultdict(list)
    for cluster_id, lines in h_lines.items():
        merged_lines[cluster_id].extend(lines)
    for cluster_id, lines in v_lines.items():
        merged_lines[cluster_id].extend(lines)
    flattened_lines = []
    for lines_in_cluster in merged_lines.values():
        for line in lines_in_cluster:
            flattened_lines.append(line)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return walls,flattened_lines
def preprocess_mask(mask):
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
    return opening

def draw_lines(mask,imgLines,EXT):
    if EXT==True:
            # line_image,lines=ext_line_walls(imgLines.copy(),line_thickness=3)
            line_image,lines=ext_line_wall(imgLines.copy(),line_thickness=3)
    else:
       if EXT==False:
            line_image,lines=int_line_walls(mask.copy(),line_thickness=3)
    return line_image,lines
def are_boundaries_adjacent(boundary1, boundary2, distance_threshold=30):
    min_edge_distance = np.min([np.min([np.linalg.norm(pt1 - pt2) for pt1 in boundary1 for pt2 in boundary2_edge]) for boundary2_edge in [boundary2, np.flip(boundary2, axis=0)]])
    return min_edge_distance < distance_threshold

### **Graph Creation**

The function ***convert_image_to_graph*** processes the image and creates a graph representation of the architectural plan. Nodes in the graph represent different zones or areas, and edges are added based on adjacency of boundaries.

**Node Creation**

> Each categorized zone from the image corresponds to a node in the graph. Nodes are created based on the color categories identified earlier. These nodes represent different areas in the architectural plan, such as rooms, corridors, and open spaces.

**Boundary-based Edge Creation**


> Edges are created between nodes based on the adjacency of their boundaries. If two zones share a boundary in the architectural plan, an edge is added between the corresponding nodes in the graph. This step establishes the spatial relationships between different areas in the plan.

 **Graph Modification**

>  Including a section for modifying the graph. Select nodes representing interior walls and changes their category to "reinforcement_concrete_interior_wall" to simulate reinforced walls.

 **Graph Representation**


> The final result of this process is a graph where nodes represent different categorized zones, and edges represent the adjacency of boundaries between these zones. This graph can be thought of as a visual abstraction of the architectural plan, where areas and their connections are captured in a structured format.

In [ ]:
def convert_image_to_graph(image_path):
    zones = []
    contours =[]
    for color, category in palette_colors.items():
        if category not in ["exterior_wall", "interior_wall"]:
            tolerance = tolerances[color]
            lower_bound = np.array(color) - np.array([tolerance * 255, tolerance * 255, tolerance * 255])
            upper_bound = np.array(color) + np.array([tolerance * 255, tolerance * 255, tolerance * 255])
            mask = cv2.inRange(cnt_img.copy(), lower_bound, upper_bound)
            outlines, _ = cv2.findContours(mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
            count = 0
            for contour in outlines:
                area = cv2.contourArea(contour)
                if area > 30:  # Filter out small noise regions
                    moments = cv2.moments(contour)
                    count += 1
                    if moments["m00"] != 0:
                        cx = int(moments["m10"] / moments["m00"])
                        cy = int(moments["m01"] / moments["m00"])
                        zones.append({
                            'category': f"{category} {count}" if count > 1 else category,
                            'area': area,
                            'centroid': (cx, cy),
                            'boundary': contour
                        })
                    cv2.drawContours(cnt_img, [contour], -1, (0, 255, 0), 1)
                    contours.append(contour)
        else:
            if category in ["exterior_wall", "interior_wall"]:
                tolerance = tolerances[color]
                lower_bound = np.array(color) - np.array([tolerance * 255, tolerance * 255, tolerance * 255])
                upper_bound = np.array(color) + np.array([tolerance * 255, tolerance * 255, tolerance * 255])
                mask = cv2.inRange(cnt_img.copy(), lower_bound, upper_bound)
                masks = {label: np.zeros_like(image)[:, :, 0] for label in palette_colors.values()}
                masks[category] += mask
                result = cv2.bitwise_and(cnt_img, cnt_img, mask=mask[:, :, np.newaxis])
                # accumulated_mask += mask
                if category in {"exterior_wall"}:
                    result = cv2.bitwise_and(cnt_img, cnt_img, mask=mask[:, :, np.newaxis])
                    filtered_mask = preprocess_mask(result)
                    # cv2.imshow(f"{category} mask",mask)
                    line_mask,ext_lines = draw_lines(filtered_mask,cnt_img.copy(),True)
                    count = 0
                    for contour in ext_lines:
                        area = cv2.contourArea(contour)
                        if area > 30: 
                            moments = cv2.moments(contour)
                            count += 1
                            if moments["m00"] != 0:
                                cx = int(moments["m10"] / moments["m00"])
                                cy = int(moments["m01"] / moments["m00"])
                                zones.append({
                                    'category': f"{category} {count}" if count > 1 else category,
                                    'area': area,
                                    'centroid': (cx, cy),
                                    'boundary': contour
                                })
                            cv2.drawContours(cnt_img, [contour], -1, (255, 0, 0), 1)
                            contours.append(contour)
                else:
                    if category == 'interior_wall':
                        filtered_mask = preprocess_mask(result)
                        line_mask, int_lines = draw_lines(filtered_mask, cnt_img.copy(),False)
                        # cv2.imshow(f"{category} lines", line_mask)
                        # print(len(int_lines))
                        count = 0
                        for line in int_lines:
                            x1,y1,x2,y2=line
                            line_length = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
                            area = line_length*2
                            # print(area)
                            if area > 30: 
                                count += 1
                                centroid_x = (x1 + x2) / 2
                                centroid_y = (y1 + y2) / 2
                                zones.append({
                                    'category': f"{category} {count}" if count > 1 else category,
                                    'area': area,
                                    'centroid': (centroid_x, centroid_y),
                                    'boundary': line
                                })
                                cv2.line(cnt_img, (x1, y1), (x2, y2), (100, 10, 200),thickness= 2,lineType=cv2.LINE_AA)
                                contours.append(line)
    # Create a graph
    graph = nx.Graph()
    interior_wall_nodes = []
    terrace_nodes=[]
    for zone in zones:
        centroid = zone['centroid']
        category = zone['category']
        area = zone['area']
        boundary = zone['boundary']
        node_color = tuple(val / 255.0 for val in category_colors[category.split()[0]])
        graph.add_node(centroid, category=category, area=area, boundary=boundary, color=node_color,centroid=centroid)

        if category.startswith('interior_wall'):
            # print("done")
            interior_wall_nodes = [
                node for node, data in graph.nodes(data=True)
                if data.get('category', '').startswith('interior_wall')
            ]
            # print(interior_wall_nodes)
            if interior_wall_nodes:
                largest_area_node = max(interior_wall_nodes, key=lambda node: graph.nodes[node].get('area', 0))
                interior_wall_nodes.remove(largest_area_node)
    terrace_node = [
        node for node, data in graph.nodes(data=True)
        if data.get('category', '').startswith('Terrace')
    ]
    terrace_nodes.extend(terrace_node)
    for terrace in terrace_nodes:
        terrace_position = terrace
        for interior_wall_node in interior_wall_nodes:
            interior_wall_position = np.array(graph.nodes[interior_wall_node]['centroid'])
            distance = math.sqrt((terrace_position[0] - interior_wall_position[0])**2 + (terrace_position[1] - interior_wall_position[1])**2)
            if distance <= 100:  # Adjust the distance threshold as needed
                interior_wall_nodes.remove(interior_wall_node)
                # print("done2")
    nodes_to_modify = interior_wall_nodes  # This assumes that interior_wall_nodes contains the modified node IDs
    # Calculate the number of nodes to select for modification (80% of the total)
    num_nodes_to_modify = int(len(nodes_to_modify) * 0.2)
    selected_indices = np.random.choice(len(nodes_to_modify), size=num_nodes_to_modify, replace=True)
    # Get the corresponding node IDs for modification
    selected_nodes = [nodes_to_modify[i] for i in selected_indices]
    for node in selected_nodes:
        graph.nodes[node]['category'] = 'reinforcement_concrete_interior_wall'
        graph.nodes[node]['color'] = (1.0, 0.0, 0.0)  # Red color
    for zone in zones:
        centroid = zone['centroid']
        category = zone['category']
        area = zone['area']
        boundary = zone['boundary']
        if category == 'entrance':
            entrance_node =next(node for node, data in graph.nodes(data=True)if data.get('category').startswith('entrance'))
            corridor_nodes = [node for node, data in graph.nodes(data=True) if data.get('category').startswith('corridor')]
            closest_corridor_node = min(corridor_nodes, key=lambda corridor_node: np.linalg.norm(np.array(graph.nodes[entrance_node].get('centroid', (0, 0))) - np.array(graph.nodes[corridor_node].get('centroid', (0, 0)))))
            ext_wall_node = next(node for node, data in graph.nodes(data=True) if data.get('category').startswith('exterior_wall'))
            graph.add_edge(entrance_node, closest_corridor_node)
            graph.add_edge(entrance_node, ext_wall_node)
            entrance_boundary = graph.nodes[entrance_node]['boundary']
            exterior_boundary = graph.nodes[ext_wall_node]['boundary']
            entrance_polygon = Polygon([(pt[0][0], pt[0][1]) for pt in entrance_boundary])
            given_entrance_polygon = LinearRing(entrance_polygon.exterior.coords)
            exterior_polygon = Polygon([(pt[0][0], pt[0][1]) for pt in exterior_boundary])
            given_exterior_polygon = LinearRing(exterior_polygon.exterior.coords)
            offset_distance = 1.4  
            offset_entrance_boundary = scale(given_entrance_polygon, xfact=  offset_distance, yfact= offset_distance)
            offset_exterior_boundary = scale(given_exterior_polygon, xfact=  offset_distance, yfact=  offset_distance)
            ent_centroid= given_entrance_polygon.centroid
            nodes_to_add = []
            scaled_boundaries = []
            all_boundary_coords = []
            exterior_coords = []
            count=1
            for i, (node1, data1) in enumerate(graph.nodes(data=True)):
                category1 = data1['category']
                # category2 = data1['category'].startswith('entrance', 'interior_wall', 'exterior_wall')
                if category1 != 'entrance' and 'wall' not in category1:
                    for j, (node2, data2) in enumerate(graph.nodes(data=True)):
                        category2 = data2['category']
                        if i != j and category2 != 'entrance' and 'wall' not in category2:
                            boundary1 = data1['boundary']
                            boundary2 = data2['boundary']
                            boundary1_list = [(pt[0][0], pt[0][1]) for pt in boundary1]
                            boundary2_list = [(pt[0][0], pt[0][1]) for pt in boundary2]
                            linear_ring1 = LinearRing(boundary1_list)
                            linear_ring2 = LinearRing(boundary2_list)
                            offset_distance = 17
                            offset_linear_ring1 = scale(linear_ring1, xfact=1 + offset_distance/100, yfact=1 + offset_distance/100)
                            offset_linear_ring2 = scale(linear_ring2, xfact=1 + offset_distance/100, yfact=1 + offset_distance/100)
                            intersectin=offset_linear_ring1.intersects(offset_linear_ring2)
                            if intersectin or offset_linear_ring1.contains_properly(offset_linear_ring2):
                                graph.add_edge(node1, node2,type='direct connection',weight=0.5)
                    offset = -0.0001
                    zone_boundary= data1['boundary']
                    # print(zone_boundary)
                    boundary_polygon = Polygon([(pt[0][0], pt[0][1]) for pt in zone_boundary])
                    given_entrance_polygon = LinearRing(boundary_polygon.exterior.coords)
                    scaled_boundary = given_entrance_polygon.offset_curve(offset,join_style='mitre',mitre_limit=0.01)
                    scaled_boundaries.append(scaled_boundary)
            merged_boundary = unary_union(scaled_boundaries)
            if isinstance(merged_boundary, MultiLineString):
                merged_polygon = merged_boundary.buffer(9,cap_style='flat',join_style='mitre',mitre_limit=0.05)
            else:
                merged_polygon = Polygon(merged_boundary)
            graph.nodes[ext_wall_node]['boundary'] = merged_polygon.exterior
            given_entrance_polygon=merged_polygon.exterior
            if ext_wall_node is not None:
                ext_wall_boundary = graph.nodes[ext_wall_node]['boundary']
                for i in range(len(ext_wall_boundary.coords) - 1):
                    start_point = ext_wall_boundary.coords[i]
                    end_point = ext_wall_boundary.coords[i + 1]
                    segment_length = np.linalg.norm(np.array(start_point) - np.array(end_point))
                    if segment_length > 40:
                        center_point = ((start_point[0] + end_point[0]) / 2, (start_point[1] + end_point[1]) / 2)
                        # opening_node = len(graph.nodes)
                        graph.add_node(center_point, category=f"opening {count}",boundary=[center_point],area=7*200,centroid=center_point)
                        count+=1
                offset_exterior_boundary = ext_wall_boundary.offset_curve(20,join_style='mitre',mitre_limit=1)
                projected_distance =    offset_exterior_boundary.project(ent_centroid)
                projected_point = offset_exterior_boundary.interpolate(projected_distance)
                projected_point = projected_point.centroid
            graph.nodes[ext_wall_node]['centroid'] = (projected_point.x,projected_point.y)
    opening_nodes= [node for node, data in graph.nodes(data=True) if data.get('category').startswith('opening')]
    for node in opening_nodes:
        graph.add_edge(node, ext_wall_node,type='fixed connection',weight=1)
            # print(graph.nodes)
    return graph,opening_nodes

# **Finding Best WiFi Spot**

***Optimize the placement of WiFi hotspots using graph theory and simulated annealing.***


using a simulated annealing solver from the ***dimod*** library to optimize the placement of WiFi hotspots in the graph. The goal is to find the best location for WiFi hotspots that satisfies certain constraints and objectives.




### **Defining zone categories, constrains and factors**




> **Wi-Fi Spot Suitability**: place the Wi-Fi spot in a zone that is suitable. This suitability can be represented by a constraint value assigned to each zone.

> **Proximity to Openings**: prefer the Wi-Fi spot to be close to opening like windows and terraces. calculate a value representing the proximity of a zone to opening.

> **Avoiding Interior Walls**: avoid placing the Wi-Fi spot near concrete interior walls. assign a penalty value to zones that are close to such walls.

> **Exclusion of Zones**: Certain zones like bathrooms, kitchens, and terraces should be excluded. assign a very high penalty value to these zones.

In [ ]:
# Define your zone categories
zone_categories = {
    "corridor","Terrace", "opening", "kitchen", "Bedroom", "Bathroom", "store",
    "entrance", "exterior_wall", "interior_wall", "reinforcement_concrete_interior_wall"
}
suitability = {
    "corridor": 0.8,
    "Terrace": 0.0, 
    "opening": 0.9,
    "kitchen": 0.0,  
    "Bedroom": 0.6,
    "Bathroom": 0.0,  
    "store": 0.3,
    "entrance": 0.8,
    "exterior_wall": 0.0,  
    "interior_wall": 0.0,  
    "reinforcement_concrete_interior_wall": 0.0
}
proximity = {
    "corridor": 0.7,
    "Terrace": 0.6,
    "opening": 5.0,
    "kitchen": 0.0,
    "Bedroom": 0.5,
    "Bathroom": 0.1,
    "store": 0.4,
    "entrance": 0.7,
    "exterior_wall": 0.5,  
    "interior_wall": 0.0,  
    "reinforcement_concrete_interior_wall": 0.0
}
wall_penalty = {
    "corridor": 0.1,
    "Terrace": 0.6,
    "opening": 0.2,
    "kitchen": 0.3,
    "Bedroom": 0.1,
    "Bathroom": 0.8,
    "store": 0.4,
    "entrance": 0.2,
    "exterior_wall": 1.0,
    "interior_wall":1.0,
    "reinforcement_concrete_interior_wall": 100
}
exclusion_penalty = {
    "corridor": 0.0,
    "Terrace": 0.0, 
    "opening": 0.0,
    "kitchen": 1.0,  
    "Bedroom": 0.0,
    "Bathroom": 1.0,  
    "store": 1.0,
    "entrance": 0.0,
    "exterior_wall": 1.0,  
    "interior_wall": 1.0,  
    "reinforcement_concrete_interior_wall": 100
}
category_to_value = {
    "corridor": 1,
    "Terrace": 2,
    "kitchen": 3,
    "Bedroom": 4,
    "Bathroom": 5,
    "store": 6,
    "entrance": 7,
    "exterior_wall": 8,
    "interior_wall": 9,
    "reinforcement_concrete_interior_wall": 10,
    "opening": 11
}
constraints = {}
for zone_category in suitability.keys():
    constraints[('suitability', zone_category)] = suitability[zone_category]
    constraints[('proximity', zone_category)] = proximity[zone_category]
    constraints[('wall_penalty', zone_category)] = wall_penalty[zone_category]
    constraints[('exclusion_penalty', zone_category)] = exclusion_penalty[zone_category]


### **Optimization**

The function ***find_best_wifi_spot*** performs simulated annealing optimization using the Dynex Annealing Sampler for finding the best location to place a WiFi hotspot on the floor plan.

In [ ]:
import pickle

def find_best_wifi_spot(bqm, graph):
    exterior_wall_node = [node for node, data in graph.nodes(data=True) if data['category'] == 'exterior_wall'][0]
    initial_x_coord = graph.nodes[exterior_wall_node]['centroid'][0]
    initial_y_coord = graph.nodes[exterior_wall_node]['centroid'][1]

    # Solve the BQM using a DynexBQM sampler (simulated annealing)
    
    sampler = SimulatedAnnealingSampler()
    response = sampler.sample(bqm, num_reads=1000)
    best_sample = response.first.sample
    print('SA RESULT:',best_sample);
    
    
    '''    
    # Solve with Dynex Platform
    model = dynex.BQM(bqm);
    sampler = dynex.DynexSampler(model,  mainnet=True, description='SamRahmeh-OptimalWiFi-HotspotPositioning')
    sampleset = sampler.sample(num_reads=10000, annealing_time = 1000, debugging=True);
    '''
    
    # optional, less optimal reads (but feasible) can also be chosen:
    '''
    energy = 1e9;
    best_sample = [];
    for sample in sampleset:
        if max([float(i) for i in sample['sample']])>0.0 and sample['energy'] < energy and sample['energy']>0:
            energy = sample['energy'];
            best_sample = sample['sample'];
            print('DEBUG: best sample taken with energy',sample['energy']);
    
    sample = {};
    i = 0;
    for var in sampler.var_mappings:
        sample[var] = 1;
        if (float(best_sample[i])<0):
            sample[var] = 0;
        i = i + 1
    
    best_sample = dimod.SampleSet.from_samples_bqm(sample, bqm).first.sample;
    '''
    
    
    '''
    best_sample = sampler.dimod_assignments.first.sample;
    print('DYNEX RESULT:',best_sample)
    '''
    best_x_coord = {node: best_sample['x_coord_{}'.format(node)] for node in graph.nodes()}
    best_y_coord = {node: best_sample['y_coord_{}'.format(node)] for node in graph.nodes()}
    dist_x = {node: initial_x_coord - best_x_coord[node] for node in graph.nodes()}
    dist_y = {node: initial_y_coord - best_y_coord[node] for node in graph.nodes()}

    max_distance_threshold = 15 
    penalty_x = {node: abs(dist) if abs(dist) > max_distance_threshold else 0 for node, dist in dist_x.items()}
    penalty_y = {node: abs(dist) if abs(dist) > max_distance_threshold else 0 for node, dist in dist_y.items()}
    for node, penalty in penalty_x.items():
        bqm.add_linear('x_coord_{}'.format(node), penalty)
    for node, penalty in penalty_y.items():
        bqm.add_linear('y_coord_{}'.format(node), penalty)
    sum_x = 0
    sum_y = 0
    count = 0
    for node, value in best_sample.items():
        if value == 1 and 'x_coord_' in node:
            category = node.split('_')[1]
            if category not in {"kitchen", "Bathroom", "reinforcement_concrete_interior_wall"}:
                coords = node.split('_')[2].strip("()")
                x_coord, y_coord = map(float, coords.split(','))
                sum_x += x_coord
                sum_y += y_coord
                count += 1
    if count == 0:
        count = 1;
    average_x = sum_x / count
    average_y = sum_y / count
    x_coords = average_x
    y_coords = average_y
    return best_sample, x_coords, y_coords

### **Wifi_hotspot Visualization**

In [ ]:
def visualize_arch_plan_with_wifi(image_path):
    graph, opening_nodes = convert_image_to_graph(image_path)
    # Initialize BQM
    bqm = BinaryQuadraticModel.empty(vartype=BINARY)
    for node in graph.nodes():
        bqm.add_variable('x_coord_{}'.format(node), 0.0)
        bqm.add_variable('y_coord_{}'.format(node), 0.0)
    print(bqm.variables)
    exterior_wall_centroid = None
    for node, data in graph.nodes(data=True):
        if data['category'] == 'exterior_wall':
            exterior_wall_node = node
            exterior_wall_centroid = data['centroid']
            initial_x_coord = data['centroid'][0]
            initial_y_coord = data['centroid'][1]
            break
    for node, data in graph.nodes(data=True):
        category = ''.join(filter(lambda x: not x.isdigit(), data['category'])).strip()
        linear_coefficient = category_to_value.get(category.startswith(zone_category), 0.0)
        bqm.add_linear(node, linear_coefficient)
        category_name_without_numbers = ''.join([c for c in data['category'] if not c.isdigit()])
        category_key = next((key for key in suitability.keys() if category_name_without_numbers in key), None)
        if category_key is not None:
            linear_coefficient = suitability[category_key]
            dist_x = initial_x_coord - data['centroid'][0]
            dist_y = initial_y_coord - data['centroid'][1]
            coeff_x = (
                linear_coefficient +
                proximity[category_key] -
                wall_penalty[category_key] -
                exclusion_penalty[category_key] -
                dist_x
            )
            coeff_y = (
                linear_coefficient +
                proximity[category_key] -
                wall_penalty[category_key] -
                exclusion_penalty[category_key] -
                dist_y
            )
            bqm.add_linear(node, coeff_x); 
            bqm.add_linear(node, coeff_y);

        else:
            linear_coefficient = 0.0
            dist_x = 0.0
            dist_y = 0.0
    
    # sampling:
    best_node,best_x_coord, best_y_coord= find_best_wifi_spot(bqm, graph)
    
    node_labels = {node: data['category'] for node, data in graph.nodes(data=True)}
    node_sizes = [data['area'] * 0.01 for node, data in graph.nodes(data=True)]
    node_colors = [tuple(val / 255.0 for val in category_colors[data['category'].split()[0]]) for node, data in graph.nodes(data=True)]
    pos = {node: data['centroid'] for node, data in graph.nodes(data=True)} # Use the centroid as the node position


    # Visualize the graph
    plt.figure(figsize=(10,10))
    # nx.draw(graph, pos, node_size=node_sizes, node_color=node_colors, labels=node_labels, font_size=5)
    nodes=graph.nodes()
    subgraph_nodes = [node for node, data in graph.nodes(data=True) if node not in opening_nodes]
    subgraph = graph.subgraph(subgraph_nodes)
    pos_subgraph = {node: data['centroid'] for node, data in subgraph.nodes(data=True) if node in subgraph_nodes}
    nx.draw_networkx_nodes(subgraph, pos, nodelist=nodes, node_size=node_sizes, node_color=node_colors,node_shape='o',edgecolors='black')
    nx.draw_networkx_labels(subgraph, pos, labels=node_labels, font_size=4, font_color='black',verticalalignment='bottom')
    edges_to_draw = subgraph.edges()
    nx.draw_networkx_edges(graph, pos_subgraph, edgelist=edges_to_draw,style='dashed',alpha=0.6)
    best_node_str = ",".join([f"{key}:{value}" for key, value in best_node.items()])
    graph.add_node(best_node_str, category="best_wifi_spot")
    plt.scatter(best_x_coord, best_y_coord, s=100, c='green', marker='o', edgecolors='black')
    plt.axis('auto')

    plt.imshow(cnt_img)
    plt.axis('off')
    plt.show()

# **Analysis Workflow**

### the GREEN_DOT is the optimal position for the WiFi-Hotspot

In [ ]:
%%time
images_folder = "dataset/"
# images_folder = "datasets/FPOP-dataset/OWHP"
# Iterate through each plan for computing
for filename in os.listdir(images_folder):
    if filename.endswith((".png", ".jpg")):
        image_path = os.path.join(images_folder, filename)
        image=cv2.imread(image_path)
        img = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2RGB)
        cnt_img =cv2.cvtColor(img.copy(), cv2.COLOR_BGR2RGB)
        visualize_arch_plan_with_wifi(image_path)
        #input("Press Enter to continue...")
